## Shahnameh Word-Level Text Generation

This project focuses on creating a text generation model inspired by the Shahnameh, an epic poem by the Persian poet Ferdowsi. By utilizing natural language processing (NLP) techniques, the model generates text at the word level, aiming to replicate the intricate style and rich content of the original work. The model is trained on the Shahnameh's extensive corpus to understand its language patterns and themes. 

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '20'

import numpy as np
import tensorflow as tf
import re
import string
import time
import keras
from keras.layers import GRU, Dropout, Dense, Embedding, TextVectorization

### Load Dataset

In [2]:
path = 'archive/shahname_fa.txt'
def read_text(path):
  with open(path, 'rb') as file:
    return file.read().decode(encoding='utf-8')

In [3]:
text = read_text(path)
text[:300]

'|به نام خداوند جان و خرد\n|کزین برتر اندیشه برنگذرد\n|خداوند نام و خداوند جای\n|خداوند روزی ده رهنمای\n|خداوند کیوان و گردان سپهر\n|فروزنده ماه و ناهید و مهر\n|ز نام و نشان و گمان برترست\n|نگارندهٔ بر شده پیکرست\n|به بینندگان آفریننده را\n|نبینی مرنجان دو بیننده را\n|نیابد بدو نیز اندیشه راه\n|که او برتر از نا'

In [4]:
len(text)

2653849

## Preprocessing

- **Custom Standardization:** Removes zero-width non-joiners, adds spaces around pipes (|), and replaces newlines with <newline>.

- **Max Tokens:** Limits the number of tokens to MAX_TOKEN, set at 19,500.

- **TextVectorization Layer:** Configured to use the custom standardization, output integer tokens, and limit the number of tokens.

In [5]:
def custom_standardization(text):
    text = tf.strings.regex_replace(text, '\u200c', '')
    text = tf.strings.regex_replace(text, r'\|', ' | ')
    text = tf.strings.regex_replace(text, '\n', ' <newline> ')
    return text

In [6]:
MAX_TOKEN = 19500
text_vec = TextVectorization(
    standardize=custom_standardization,
    output_mode="int",
    max_tokens=MAX_TOKEN
)

text_vec.adapt(tf.constant([text]))


In [7]:
text_vec.get_vocabulary()

['',
 '[UNK]',
 '|',
 '<newline>',
 'و',
 'به',
 'که',
 'ز',
 'از',
 'بر',
 'را',
 'چو',
 'با',
 'گفت',
 'شد',
 'تو',
 'شاه',
 'همی',
 'بود',
 'او',
 'یکی',
 'همه',
 'آن',
 'من',
 'اندر',
 'در',
 'تا',
 'سر',
 'چنین',
 'کرد',
 'آمد',
 'دل',
 'پیش',
 'این',
 'جهان',
 'بد',
 'پر',
 'بدو',
 'هر',
 'سپاه',
 'چون',
 'داد',
 'پس',
 'دو',
 'نه',
 'سوی',
 'راه',
 'کار',
 'لشکر',
 'روی',
 'چه',
 'تخت',
 'سخن',
 'ما',
 'مرد',
 'مرا',
 'گر',
 'جای',
 'هم',
 'زمین',
 'گرد',
 'جنگ',
 'دست',
 'ایران',
 'گشت',
 'بیامد',
 'همان',
 'روز',
 'باد',
 'یک',
 'کسی',
 'شهریار',
 'بدین',
 'بدان',
 'خویش',
 'ازان',
 'کس',
 'تاج',
 'اگر',
 'رای',
 'ای',
 'تن',
 'اوی',
 'گنج',
 'بران',
 'خون',
 'نام',
 'کنون',
 'نزدیک',
 'زین',
 'آفرین',
 'نیست',
 'رنج',
 'آب',
 'برو',
 'کوه',
 'نیز',
 'کین',
 'چنان',
 'شود',
 'پاسخ',
 'رستم',
 'اندرون',
 'زان',
 'خرد',
 'دید',
 'گیتی',
 'شیر',
 'یزدان',
 'آید',
 'میان',
 'دگر',
 'شب',
 'ترا',
 'جز',
 'بهرام',
 'برین',
 'جان',
 'شاد',
 'درد',
 'کجا',
 'خاک',
 'بسی',
 'دشت',
 'سپ

### Converting Token IDs to Text: 

- The ids_to_text function converts a sequence of token IDs back into readable text using the vocabulary from the TextVectorization layer.
- It maps each token ID to its corresponding word in the vocabulary, replaces <newline>| tokens with actual newline characters, and then joins the words into a single string.

In [8]:
def ids_to_text(ids):
    text = [text_vec.get_vocabulary()[idx] for idx in ids]
    text = tf.strings.regex_replace(text, '<newline>', '\n')
    return tf.strings.reduce_join(text, separator=' ').numpy().decode('UTF-8')

In [9]:
ids = [18, 45, 51, 25, 3, 8, 9]
print(ids_to_text(ids))

بود سوی تخت در 
 از بر


## Create training examples and targets

- Next, divide the text into example sequences. Each input sequence will contain `seq_length` tokens from the text.

- For each input sequence, the corresponding targets contain the same length of text, except shifted one token to the right.

- To achieve this, break the text into chunks of `seq_length + 1`. For instance, if seq_length is 4 and our text is "Hello", the input sequence would be "Hell", and the target sequence would be "ello".

In [10]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
MAX_SEQ_LEN = 10

def vectorizer(text):
    return text_vec(text)

dataset = tf.data.Dataset.from_tensor_slices(tf.constant([text]))
dataset = dataset.cache().map(vectorizer)

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text


dataset = dataset.flat_map(lambda x: tf.data.Dataset.from_tensor_slices(x))
seq = dataset.batch(MAX_SEQ_LEN + 1, drop_remainder=True)

dataset = seq.map(split_input_target)


In [11]:
for input_ids, target_ids in dataset.take(5):
    print("Input:", ids_to_text(input_ids))
    print("Target:", ids_to_text(target_ids))

Input: | به نام خداوند جان و خرد 
 | کزین
Target: به نام خداوند جان و خرد 
 | کزین برتر
Input: اندیشه برنگذرد 
 | خداوند نام و خداوند جای 

Target: برنگذرد 
 | خداوند نام و خداوند جای 
 |
Input: خداوند روزی ده رهنمای 
 | خداوند کیوان و گردان
Target: روزی ده رهنمای 
 | خداوند کیوان و گردان سپهر
Input: 
 | فروزنده ماه و ناهید و مهر 
 |
Target: | فروزنده ماه و ناهید و مهر 
 | ز
Input: نام و نشان و گمان برترست 
 | نگارندهٔ بر
Target: و نشان و گمان برترست 
 | نگارندهٔ بر شده


## Create training batches

After segmenting the text into manageable sequences, the next step is to prepare the data for model training. This involves two main steps: shuffling the data and packing it into batches.

Shuffling ensures that the model encounters a diverse range of examples during training, preventing it from learning any sequential patterns. Batching groups these sequences together, making training more efficient by processing multiple examples at once.



In [12]:
BATCH_SIZE = 64
SHUFFLE_BUFFER = 1000

dataset = dataset.shuffle(SHUFFLE_BUFFER)
dataset = dataset.batch(batch_size=BATCH_SIZE, num_parallel_calls=AUTOTUNE)
dataset = dataset.prefetch(AUTOTUNE)

In [13]:
for input_ids, target_ids in dataset.take(1):
    print("Input:", input_ids)
    print("Target:", target_ids)

Input: tf.Tensor(
[[   21   420 19286   129  1254    29     3     2    11    12]
 [    5    32   756    22  1135     3     2  1246     4  2373]
 [ 1872   450  1158     3     2    76     8  3921   451   895]
 [   21  5887  2493     8  8536     3     2    20 16443     5]
 [ 3712   598   458     3     2    14    22   532  2392 18665]
 [  438    91     3     2     6   239     4   973  1851  1316]
 [    3     2    11   144   587  1744    10  3301     3     2]
 [    3     2   863    29   760     4  2289     4  2087     3]
 [   77   155    51     3     2 15249   422    26  1894     3]
 [   18  1925     4  5884     3     2   218     4   244     4]
 [ 2076    90   212   143    90     3     2     9    22   168]
 [  106   895  8333   267     3     2    17 12076    12  2412]
 [    3     2   371     8   432  6279    47    29     3     2]
 [    2    17    13    12    38    70    79    74     3     2]
 [   49     3     2     5   286    94    90   493     3     2]
 [   18     3     2    36     8  1129

## Model

The **MyModel** class is a custom TensorFlow/Keras model for text generation. It comprises three main layers:

- **Embedding Layer:** Maps input token indices to dense embedding vectors.
- **GRU Layer:** Processes the embedded sequences, producing output sequences and new states.
- **Dense Layer:** Converts GRU output sequences to logits over the vocabulary.

The `call` method defines the forward pass of the model, taking input sequences and optionally initial states, and returning output logits and new states if specified.

In [14]:
class MyModel(keras.models.Model):
    def __init__(self, vocab_size, embd_dim, rnn_units):
        super(MyModel, self).__init__()
        
        self.embedding = Embedding(vocab_size, embd_dim)
        self.gru = GRU(rnn_units, return_sequences=True, return_state=True)
        self.dense = Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        
        x = self.embedding(inputs)
        
        if states == None:
            states = self.gru.get_initial_state(x)
            
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x)
        if return_state:
            return x, states
        else:
            return x


In [15]:
VOCAB_SIZE = text_vec.vocabulary_size()
EMBD_DIM = 512
RNN_UNITS = 1048
model = MyModel(VOCAB_SIZE, EMBD_DIM, RNN_UNITS)

In [16]:
for input_ids, target_ids in dataset.take(1):
    pred = model(input_ids)
    print(pred.shape)

(64, 10, 19500)


In the above example the sequence length of the input is 100 but the model can be run on inputs of any length

In [17]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  9984000   
                                                                 
 gru (GRU)                   multiple                  4910928   
                                                                 
 dense (Dense)               multiple                  20455500  
                                                                 
Total params: 35350428 (134.85 MB)
Trainable params: 35350428 (134.85 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Train the model

In [18]:
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [19]:
model.fit(dataset, epochs=30)

Epoch 1/30


I0000 00:00:1717410900.754582    6651 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1081/1081 [==============================] - 31s 28ms/step - loss: 5.0449
Epoch 2/30
1081/1081 [==============================] - 29s 26ms/step - loss: 4.0436
Epoch 3/30
1081/1081 [==============================] - 29s 26ms/step - loss: 3.5167
Epoch 4/30
1081/1081 [==============================] - 29s 26ms/step - loss: 3.0462
Epoch 5/30
1081/1081 [==============================] - 29s 26ms/step - loss: 2.6190
Epoch 6/30
1081/1081 [==============================] - 29s 26ms/step - loss: 2.2516
Epoch 7/30
1081/1081 [==============================] - 29s 26ms/step - loss: 1.9414
Epoch 8/30
1081/1081 [==============================] - 29s 26ms/step - loss: 1.6866
Epoch 9/30
1081/1081 [==============================] - 29s 26ms/step - loss: 1.4837
Epoch 10/30
1081/1081 [==============================] - 29s 26ms/step - loss: 1.3207
Epoch 11/30
1081/1081 [==============================] - 29s 27ms/step - loss: 1.1992
Epoch 12/30
1081/1081 [==============================] - 29s 26ms/step - l

## Text Genaration

Generating text with a word-level model involves iterating through a loop to predict the next word based on the preceding words. 

It starts with an `initial text`, an initial phrase or sentence. This initial text is fed into the model, which then predicts the subsequent word. The predicted word is appended to the initial text, and the process repeats. Each iteration adds one word to the generated text, gradually forming a coherent sequence. This iterative approach continues until the desired length of text is generated or until a stopping condition is met. By predicting words based on context, the model learns to produce text that mirrors the style and content of the training data, resulting in coherent and meaningful passages.


![Alt text](download.png)


In [23]:
class One_Step(keras.models.Model):
    def __init__(self, model, text_vec, temperature=1.0):
        super(One_Step, self).__init__()
        self.model = model
        self.text_vec = text_vec
        self.temperature = temperature
        
        skip_ids = tf.constant([text_vec.get_vocabulary().index('[UNK]')], dtype=tf.int64)
        skip_ids = tf.reshape(skip_ids, [-1, 1])
        
        sparse_mask = tf.SparseTensor(
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        dense_shape=[text_vec.vocabulary_size()])
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function()
    def generate_one_step(self, inputs, states=None):

        inputs = tf.constant([inputs])
        ids = self.text_vec(inputs)
        predicted, states = model(inputs=ids, states=states, return_state=True)
        predicted = predicted[:, -1, :]
        predicted = predicted / self.temperature
        predicted = predicted + self.prediction_mask

        predicted_id = tf.random.categorical(predicted, num_samples=1)
        predicted_id = tf.squeeze(predicted_id, axis=-1)
        
        return predicted_id, states

## Text Generation Using One Step Model

This code generates text using the one-step text generation model (one_step_model).

It initializes with a starting phrase, `"به نام خداوند"` (In the name of God). Then, it iteratively calls the generate_one_step method of the model to predict the next word based on the previous words. The generated words are appended to the result list. After generating the desired number of words (300 in this case).

In [34]:
one_step_model = One_Step(model, text_vec, temperature = 0.9)

In [39]:
start = time.time()
states = None
next_char = ' به نام خداوند '
result = [next_char]

for n in range(300):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    next_char = ids_to_text(next_char)
    result.append(next_char)

result = tf.strings.join(result, separator=' ')
end = time.time()
print(result.numpy().decode('utf-8'), '\n\n')
print('\nRun time:', end - start)


 به نام خداوند  فر و مهان 
 | به بیدادگر بر جهان شهریار 
 | که آمد که پیروز باد مرا 
 | به خورشید پیمان که آمد به ماه 
 | که آباد تاج پدر یادگار 
 | ز جمشید بر هر سویی شاد دار 
 | چو کم شد به ایران ز فرزند شاه 
 | که این بد که بر کوی تن شهریار 
 | به گیتی چنین بود زرد و به روز 
 | مرا دل بر و گیتی پراگنده گشت 
 | بگفت این و دو دیده بر دست رفت 
 | چنین رفت بر دیدهبان بر بدرید خویش 
 | چنین گفت بر سان سخن راز بود 
 | به گیتی ز ترکان کسی بینیاز 
 | به من کردگار جهان بینیاز 
 | به شبگیر کی بود و آمد به جای 
 | گمان ای شما بر خرد 
 | سخن با بزرگ و به جنگ 
 | سخن گر برو بر لب جویبار 
 | کمان را چو گرد کردم به دشت 
 | بزد کوس و شکسته سواری آورید 
 | زمین را کنون او سر اندر هوا 
 | چو بشنید شاه این سخن راست شد 
 | بر جهان آفرین برین باد و بر 
 | جهان یکسر اندر جهان آورید 
 | بر این بر یکی بیژن و بردش نماز 
 | یکی تیر بر داده جان او تیر 
 | یکی ابر و بر زد یکی پیشکار 
 | ابا جای بر جای بر سر گرفت 
 | بران نامور اندر اندر جهان 
 | بر اندر هوا کوه چون بر زمین 
 | هوا پر ز فرمان و دل پر ز جوش 
 | 